# Image Processing and Analysis - Assignment 01
## SCC0251 - 2023.1
### Thaís Ribeiro Lauriano - 12542518

In [39]:
import imageio.v2 as imageio
import numpy as np

In [50]:
#Reading image names and loading image files

imglow_bn = input()

img_low = []
for i in range(4):
    img_low.append(imageio.imread(imglow_bn+str(i)+'.png'))

imghigh_name = input()
img_high = imageio.imread(imghigh_name)

In [51]:
def same_shape(img1, img2):
    if img1.shape != img2.shape:
        print("Images aren't the same resolution")
        return False
    return True

In [52]:
def superresolution(imgs):
    n = imgs[0].shape
    if not same_shape(imgs[0], imgs[1]) or not same_shape(imgs[0], imgs[2]) or not same_shape(imgs[0], imgs[3]):
        return

    gen_img = np.zeros((2*n[0], 2*n[1]))

    for i in range(n[0]):
        for j in range(n[1]):
            gen_img[i*2][j*2] = imgs[0][i][j]
            gen_img[i*2+1][j*2] = imgs[1][i][j]
            gen_img[i*2][j*2+1] = imgs[2][i][j]
            gen_img[i*2+1][j*2+1] = imgs[3][i][j]
    
    return gen_img

In [53]:
def histogram(img, n_levels):
    hist = np.zeros(n_levels).astype(int)

    for i in range(n_levels):
        pixels_value_i = np.sum(img == i)
        hist[i] = pixels_value_i
            
    return(hist)

In [ ]:
def cumulative_histogram(img, n_levels):
    hist = histogram(img,  n_levels)
    
    histC = np.zeros(n_levels).astype(int)

    histC[0] = hist[0] 
    for i in range(1,  n_levels):
        histC[i] = hist[i] + histC[i-1]

    return histC

In [54]:
def histogram_equalization(img, n_levels, histC):

    n = img.shape
    
    img_eq = np.zeros([n[0],n[1]]).astype(np.uint8)
     
    for input in range(n_levels):
        output = ((n_levels-1)/float(n[0]*n[1]))*histC[input]
        
        img_eq[ np.where(img == input) ] = output
    
    return img_eq

In [55]:
def single_img_cumul_hist(imgs):
    eq_imgs = []
    for i in range(4):
        eq_imgs.append(histogram_equalization(imgs[i], 256, cumulative_histogram(imgs[i], 256)))

    return eq_imgs

In [ ]:
def joint_cumulative_hist(imgs):
    joint_imgs = np.concatenate(imgs[0], imgs[1], imgs[3], imgs[4])
    histC = cumulative_histogram(joint_imgs, 256)

    eq_imgs = []
    for i in range(4):
        eq_imgs.append(histogram_equalization(imgs[i], 256, histC))


    return eq_imgs

In [56]:
def gamma_correction(imgs, gamma):
    n = imgs[0].shape
    if not same_shape(imgs[0], imgs[1]) or not same_shape(imgs[0], imgs[2]) or not same_shape(imgs[0], imgs[3]):
        return

    for k in range(4):
        for i in range(n[0]):
            for j in range(n[1]):
                imgs[k][i][j] = 255 * (imgs[k][i][j]/255.0)**(1/gamma)

In [57]:
# RMSE
def comparison(gen_img, ref_img):
    n = gen_img.shape
    if not same_shape(gen_img, ref_img):
        return
    sum = 0
    for i in range(n[0]):
        for j in range(n[1]):
            sum += (ref_img[i][j] - gen_img[i][j])**2
            
    error = np.sqrt(sum/(n[0]*n[1]))
    print("{:.4f}".format(error))
    return

In [59]:
F = int(input())
gamma = float(input())

if F > 3 or F < 0:
    print("Invalid value of F")
elif F == 0:
    pass
elif F == 1:
    single_img_cumul_hist(img_low)
elif F == 2:
    joint_cumulative_hist(img_low)
elif F == 3:
    gamma_correction(img_low, gamma)
comparison(superresolution(img_low), img_high)

17.9062
